In [17]:
import tkinter as tk
import time
import random

# Paragraphs for typing test
EASY_TEXTS = [
    "apple, banana, orange",
    "cat, dog, fish",
    "sun, moon, star"
]

MEDIUM_TEXTS = [
    "The quick brown fox jumps over the lazy dog.",
    "Typing is fun when you practice daily.",
    "Python is easy to learn and very powerful."
]

HARD_TEXTS = [
    "Technology changes the way we live, work, and connect with others.",
    "Consistency and dedication are the keys to mastering any skill in life.",
    "A typing test is a great way to measure both speed and accuracy together."
]


class TypingTestApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Typing Speed Test")
        self.geometry("800x500")
        self.configure(bg="#1c1c1c")

        self.selected_paragraphs = []

        self.frames = {}
        for F in (StartPage, DifficultyPage, TestPage, ResultsPage):
            frame = F(self)
            self.frames[F] = frame
            frame.place(relwidth=1, relheight=1)

        self.show_frame(StartPage)

    def show_frame(self, frame_class):
        self.frames[frame_class].tkraise()


class StartPage(tk.Frame):
    def __init__(self, master):
        super().__init__(master, bg="#1c1c1c")

        tk.Label(self, text="Typing Speed Test", font=("Arial", 24, "bold"),
                 fg="white", bg="#1c1c1c").pack(pady=80)

        start_btn = tk.Button(self, text="Start", font=("Arial", 16), fg="white",
                              bg="#444", relief="flat", padx=20, pady=10,
                              command=lambda: master.show_frame(DifficultyPage))
        start_btn.pack()


class DifficultyPage(tk.Frame):
    def __init__(self, master):
        super().__init__(master, bg="#1c1c1c")

        tk.Label(self, text="Select Difficulty", font=("Arial", 20, "bold"),
                 fg="white", bg="#1c1c1c").pack(pady=50)

        btn_frame = tk.Frame(self, bg="#1c1c1c")
        btn_frame.pack()

        tk.Button(btn_frame, text="Easy", font=("Arial", 14), fg="white", bg="#2e8b57",
                  padx=20, pady=10, command=lambda: self.start_test(master, EASY_TEXTS)).grid(row=0, column=0, padx=10)

        tk.Button(btn_frame, text="Medium", font=("Arial", 14), fg="white", bg="#1e90ff",
                  padx=20, pady=10, command=lambda: self.start_test(master, MEDIUM_TEXTS)).grid(row=0, column=1, padx=10)

        tk.Button(btn_frame, text="Hard", font=("Arial", 14), fg="white", bg="#b22222",
                  padx=20, pady=10, command=lambda: self.start_test(master, HARD_TEXTS)).grid(row=0, column=2, padx=10)

        tk.Button(self, text="Exit", font=("Arial", 12), fg="white", bg="#333",
                  command=lambda: master.show_frame(StartPage)).pack(pady=20)

    def start_test(self, master, paragraphs):
        master.selected_paragraphs = paragraphs
        master.frames[TestPage].start_test()
        master.show_frame(TestPage)


class TestPage(tk.Frame):
    def __init__(self, master):
        super().__init__(master, bg="#1c1c1c")

        self.current_index = 0
        self.start_time = None
        self.results = []
        self.master_ref = master

        self.progress_label = tk.Label(self, text="", font=("Arial", 12),
                                       fg="white", bg="#1c1c1c")
        self.progress_label.pack(pady=5)

        self.paragraph_label = tk.Label(self, text="", wraplength=700, font=("Arial", 14),
                                        fg="#ddd", bg="#1c1c1c", justify="left")
        self.paragraph_label.pack(pady=10)

        self.text_entry = tk.Text(self, height=5, width=80, font=("Arial", 14),
                                  fg="#ddd", bg="#2e2e2e", wrap="word", insertbackground="white")
        self.text_entry.pack(pady=10)
        self.text_entry.bind("<KeyRelease>", self.update_live_wpm)

        self.live_wpm_label = tk.Label(self, text="WPM: 0", font=("Arial", 12),
                                       fg="white", bg="#1c1c1c")
        self.live_wpm_label.pack()

        btn_frame = tk.Frame(self, bg="#1c1c1c")
        btn_frame.pack(pady=20)

        self.next_btn = tk.Button(btn_frame, text="Next", font=("Arial", 12), fg="white",
                                  bg="#444", command=self.finish_paragraph)
        self.next_btn.grid(row=0, column=0, padx=5)

        self.finish_btn = tk.Button(btn_frame, text="Finish Test", font=("Arial", 12),
                                    fg="white", bg="#555", command=self.finish_test)
        self.finish_btn.grid(row=0, column=1, padx=5)

        self.exit_btn = tk.Button(btn_frame, text="Exit", font=("Arial", 12),
                                  fg="white", bg="#333",
                                  command=lambda: master.show_frame(StartPage))
        self.exit_btn.grid(row=0, column=2, padx=5)

    def start_test(self):
        self.current_index = 0
        self.results.clear()
        self.load_paragraph()

    def load_paragraph(self):
        self.text_entry.delete(1.0, tk.END)
        self.live_wpm_label.config(text="WPM: 0")
        self.progress_label.config(text=f"Paragraph {self.current_index + 1} of {len(self.master_ref.selected_paragraphs)}")
        self.paragraph_label.config(text=self.master_ref.selected_paragraphs[self.current_index])
        self.start_time = time.time()

    def update_live_wpm(self, event=None):
        typed_text = self.text_entry.get("1.0", tk.END).strip()
        elapsed_time = time.time() - self.start_time
        if elapsed_time > 0:
            words = len(typed_text.split())
            wpm = (words / elapsed_time) * 60
            self.live_wpm_label.config(text=f"WPM: {int(wpm)}")

    def finish_paragraph(self):
        end_time = time.time()
        typed_text = self.text_entry.get("1.0", tk.END).strip()
        elapsed_time = end_time - self.start_time
        words = len(typed_text.split())
        accuracy = self.calculate_accuracy(typed_text, self.master_ref.selected_paragraphs[self.current_index])
        wpm = (words / elapsed_time) * 60 if elapsed_time > 0 else 0

        self.results.append({
            "time": elapsed_time,
            "words": words,
            "wpm": int(wpm),
            "accuracy": int(accuracy)
        })

        self.current_index += 1
        if self.current_index < len(self.master_ref.selected_paragraphs):
            self.load_paragraph()
        else:
            self.finish_test()

    def finish_test(self):
        self.master_ref.frames[ResultsPage].display_results(self.results)
        self.master_ref.show_frame(ResultsPage)

    def calculate_accuracy(self, typed, original):
        typed_words = typed.split()
        original_words = original.split()
        correct = sum(1 for i, word in enumerate(typed_words)
                      if i < len(original_words) and word == original_words[i])
        return (correct / len(original_words)) * 100 if original_words else 0


class ResultsPage(tk.Frame):
    def __init__(self, master):
        super().__init__(master, bg="#1c1c1c")

        self.result_label = tk.Label(self, text="", font=("Arial", 14),
                                     fg="white", bg="#1c1c1c", justify="left")
        self.result_label.pack(pady=50)

        tk.Button(self, text="Restart", font=("Arial", 12), fg="white", bg="#444",
                  command=lambda: master.show_frame(StartPage)).pack()

    def display_results(self, results):
        if not results:
            self.result_label.config(text="No results recorded.")
            return
        avg_wpm = sum(r['wpm'] for r in results) / len(results)
        avg_accuracy = sum(r['accuracy'] for r in results) / len(results)
        total_time = sum(r['time'] for r in results)

        result_text = (
            f"Paragraphs completed: {len(results)}\n"
            f"Average WPM: {avg_wpm:.2f}\n"
            f"Average Accuracy: {avg_accuracy:.2f}%\n"
            f"Total Time: {total_time:.2f} seconds"
        )
        self.result_label.config(text=result_text)


if __name__ == "__main__":
    app = TypingTestApp()
    app.mainloop()
